In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cidaut-ai-fake-scene-classification-2024/sample_submission.csv
/kaggle/input/cidaut-ai-fake-scene-classification-2024/train.csv
/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test/473.jpg
/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test/537.jpg
/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test/785.jpg
/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test/239.jpg
/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test/792.jpg
/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test/109.jpg
/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test/544.jpg
/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test/576.jpg
/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test/613.jpg
/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test/243.jpg
/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test/153.jpg
/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test/143.jpg
/kaggle/input/cidaut-ai-fake-scene-classif

In [2]:
import pydotplus
import pydot
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D,Input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
import datetime
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image

# Load Data

In [3]:
#Load data
train_path='/kaggle/input/cidaut-ai-fake-scene-classification-2024/Train'
train = pd.read_csv('/kaggle/input/cidaut-ai-fake-scene-classification-2024/train.csv')
train['image']=train['image'].apply(lambda x : os.path.join(train_path,x)) 
clss={'editada':0,'real':1}
train['label']=train['label'].map(clss)

In [4]:
train.head()

,image,label
0,/kaggle/input/cidaut-ai-fake-scene-classificat...,0
1,/kaggle/input/cidaut-ai-fake-scene-classificat...,1
2,/kaggle/input/cidaut-ai-fake-scene-classificat...,1
3,/kaggle/input/cidaut-ai-fake-scene-classificat...,0
4,/kaggle/input/cidaut-ai-fake-scene-classificat...,1


read img in HsV

In [5]:
def processImgHSV(imgpath):
    img = load_img(imgpath, target_size=(224, 224))  # Load the image and resize
    img_array = img_to_array(img)  # Convert to a NumPy array
    img_array /= 255.0  # Normalize pixel values to [0, 1]
    
    # Convert from RGB to HSV using OpenCV
    img_hsv = cv2.cvtColor(np.array(img_array), cv2.COLOR_BGR2HLS)
    return img_hsv

In [6]:
def processImgHLS(imgpath):
    img = load_img(imgpath, target_size=(224, 224))  # Load the image and resize
    img_array = img_to_array(img)  # Convert to a NumPy array
    img_array /= 255.0  # Normalize pixel values to [0, 1]
    
    # Convert from RGB to HSV using OpenCV
    img_hsv = cv2.cvtColor(np.array(img_array), cv2.COLOR_BGR2HLS)
    return img_hsv

train val

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   720 non-null    object
 1   label   720 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.4+ KB


In [8]:
X=np.array([processImgHSV(imgpath) for imgpath in train['image']])
y=np.array(train['label'])


In [9]:
# Augmentation setup using Keras' ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Now you can use datagen to augment the images
datagen.fit(X)  # Fit the generator to your data

# Example: Generate batches of augmented images for training
augmented_images = datagen.flow(X, y, batch_size=32)

In [10]:
X.shape

(720, 224, 224, 3)

In [11]:
y.shape

(720,)

split data

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)


Alex model

In [13]:

# AlexNet model
class AlexNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Conv2D(96, kernel_size=(11,11), strides= 4,
                        padding= 'valid', activation= 'relu',
                        input_shape= input_shape,
                        kernel_initializer= 'he_normal'))
        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

        self.add(Conv2D(256, kernel_size=(5,5), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))
        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None)) 
        
        self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(Conv2D(256, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

        self.add(Flatten())
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dense(1000, activation= 'relu'))
        self.add(Dense(1, activation= 'sigmoid'))
        
        self.compile(optimizer= tf.keras.optimizers.Adam(0.001),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])



In [14]:
num_classes = 2
model = AlexNet((227, 227, 3), num_classes)
callbacks_list = [
    EarlyStopping(monitor='loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='best_AlexNetHSL_model.keras', monitor='loss', save_best_only=True, mode='min')
]

#balance class
#class_weights = {0: weight_for_class_0, 1: weight_for_class_1}

history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=30,
    validation_data=(X_val, y_val),
    callbacks=callbacks_list
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30


ValueError: Exception encountered when calling AlexNet.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 9216, but received input with shape (32, 6400)[0m

Arguments received by AlexNet.call():
  • inputs=tf.Tensor(shape=(32, 224, 224, 3), dtype=float32)
  • training=True
  • mask=None

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred = model.predict(X_val)
print("Validation AUC-ROC:", roc_auc_score(y_val, y_pred))

retrain with orignal data without augmentation 

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create an iterator for the training data
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)


In [ ]:
val_datagen = ImageDataGenerator()

# Create an iterator for the validation data
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)


In [ ]:
#using augmented data
modelvgg.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=len(X_train) // 32,
    validation_steps=len(X_val) // 32
)

VGG

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

modelvgg = Model(inputs=base_model.input, outputs=predictions)
modelvgg.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks_list = [
    EarlyStopping(monitor='loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='VGGHSV.keras', monitor='loss', save_best_only=True, mode='min')
]

#balance class
#class_weights = {0: weight_for_class_0, 1: weight_for_class_1}

history = modelvgg.fit(
    X_train, y_train,
    batch_size=32,
    epochs=30,
    validation_data=(X_val, y_val),
    callbacks=callbacks_list
)


In [ ]:
from sklearn.metrics import roc_auc_score

y_pred = modelvgg.predict(X_val)
print("Validation AUC-ROC:", roc_auc_score(y_val, y_pred))

In [ ]:
#using augmented data
modelvgg.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
)

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred = modelvgg.predict(X_val)
print("Validation AUC-ROC:", roc_auc_score(y_val, y_pred))

Try read data hsl

In [ ]:
X2=np.array([processImgHLS(imgpath) for imgpath in train['image']])
#y=np.array(train['label'])
X_train, X_val, y_train, y_val = train_test_split(X2, y, test_size=0.2, random_state=1)


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

modelvggHSL = Model(inputs=base_model.input, outputs=predictions)
modelvggHSL.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks_list = [
    EarlyStopping(monitor='loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='VGGHSV.keras', monitor='loss', save_best_only=True, mode='min')
]

#balance class
#class_weights = {0: weight_for_class_0, 1: weight_for_class_1}

history = modelvggHSL.fit(
    X_train, y_train,
    batch_size=32,
    epochs=30,
    validation_data=(X_val, y_val),
    callbacks=callbacks_list
)


In [ ]:
from sklearn.metrics import roc_auc_score

y_pred = modelvggHSL.predict(X_val)
print("Validation AUC-ROC:", roc_auc_score(y_val, y_pred))

Try retrain with augmented data

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create an iterator for the training data
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)


In [ ]:
val_datagen = ImageDataGenerator()

# Create an iterator for the validation data
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)


In [ ]:
callbacks_list = [
    EarlyStopping(monitor='loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='VGGHSL.keras', monitor='loss', save_best_only=True, mode='min')
]
modelvggHSL.fit(

    train_generator,
    validation_data=val_generator,
    epochs=10,
)

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred = modelvggHSL.predict(X_val)
print("Validation AUC-ROC:", roc_auc_score(y_val, y_pred))

# data augmentations 

In [ ]:
X2=np.array([processImgHLS(imgpath) for imgpath in train['image']])
#y=np.array(train['label'])
X_train, X_val, y_train, y_val = train_test_split(X2, y, test_size=0.2, random_state=1)


In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create an iterator for the training data
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)


In [ ]:
val_datagen = ImageDataGenerator()

# Create an iterator for the validation data
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

modelvggHSLA = Model(inputs=base_model.input, outputs=predictions)
modelvggHSLA.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
steps_per_epoch = len(X_train) // 32  # 720 images, batch size 32 -> 22 steps per epoch

callbacks_list = [
    EarlyStopping(monitor='loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='VGGHSLA.keras', monitor='loss', save_best_only=True, mode='min')
]
modelvggHSLA.fit(
    train_generator,  # Training data generator
    epochs=50,        # Number of epochs to train
    steps_per_epoch=steps_per_epoch,  # Number of steps per epoch
    validation_data=val_generator,  # Validation data generator
    validation_steps=len(X_val) // 32  # Steps per epoch for validation (adjust accordingly)
)

In [ ]:
# Paths to your training and validation datasets
train_dir = 'path_to_training_data'
val_dir = 'path_to_validation_data'

# Create an ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20%
    height_shift_range=0.2,  # Randomly shift images vertically by 20%
    shear_range=0.2,  # Randomly apply shearing
    zoom_range=0.2,  # Randomly zoom into images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill in pixels for rotated or shifted images
)

# Validation data generator (no augmentation for validation)
val_datagen = ImageDataGenerator(rescale=1.0/255)

# Load and augment data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize all images to 150x150
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


#agument using pytorch 

In [ ]:
# Install required libraries
!pip install torch torchvision albumentations timm pandas scikit-learn matplotlib tqdm torchmetrics opencv-python

# importing 

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
from torchvision.models import resnet50, efficientnet_v2_s
from albumentations.pytorch import ToTensorV2
import albumentations as A
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import timm

## 4. Advanced Custom Dataset Class
> Here, we integrate OpenCV for image loading and Albumentations for augmentations.

In [ ]:
class AdvancedImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        """
        Args:
            csv_file (pd.DataFrame): Pandas DataFrame içindeki dosya adları ve etiketler.
            img_dir (str): Görsellerin bulunduğu dizin.
            transform (callable, optional): Görseller için dönüşüm/augmentasyon işlemleri.
        """
        self.annotations = csv_file
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # Görsel yolunu alın
        img_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV ile BGR -> RGB
        # Etiketi alın
        label = torch.tensor(self.annotations.iloc[idx, 1], dtype=torch.float32)

        # Dönüşümleri uygulayın
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        # Görüntü formatını `float32` yapın
        image = image.float()
        
        return image, label

In [ ]:
def get_transforms(img_size):
    train_transform = A.Compose([
        A.Resize(img_size, img_size),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.GaussianBlur(p=0.1),
        A.CoarseDropout(max_holes=4, max_height=16, max_width=16, min_holes=1, min_height=8, min_width=8, fill_value=0, p=0.5),
        ToTensorV2()
    ])

    valid_transform = A.Compose([
        A.Resize(img_size, img_size),
        ToTensorV2()
    ])
    return train_transform, valid_transform

In [ ]:
class AdvancedModel(nn.Module):
    def __init__(self, num_classes=1):
        super(AdvancedModel, self).__init__()
        # EfficientNet-B0 kullanımı
        self.backbone = timm.create_model('efficientnet_b0', pretrained=True)
        in_features = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)

Training and Validation Functions¶
Automatic Mixed Precision (AMP) and advanced metrics are added.

In [ ]:
def train_one_epoch(model, train_loader, criterion, optimizer, scheduler, scaler, device):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device, dtype=torch.float32), labels.to(device, dtype=torch.float32).unsqueeze(1)

        optimizer.zero_grad()
        with torch.amp.autocast(device_type='cuda'):  # AMP için güncelleme
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        running_loss += loss.item() * images.size(0)

    return running_loss / len(train_loader.dataset)

def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_outputs = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images, labels = images.to(device, dtype=torch.float32), labels.to(device, dtype=torch.float32).unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            all_labels.append(labels.cpu().numpy())
            all_outputs.append(torch.sigmoid(outputs).cpu().numpy())

    epoch_loss = running_loss / len(val_loader.dataset)
    all_labels = np.concatenate(all_labels)
    all_outputs = np.concatenate(all_outputs)

    return epoch_loss, all_labels, all_outputs

Training Loop
Added Early Stopping and advanced metrics logging.

In [ ]:
def train_model(csv_file, img_dir, model, img_size, batch_size, lr, num_epochs, device, patience):
    # Prepare data
    train_transform, valid_transform = get_transforms(img_size)
    train_dataset = AdvancedImageDataset(csv_file, img_dir, transform=train_transform)
    val_dataset = AdvancedImageDataset(csv_file, img_dir, transform=valid_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model, optimizer, scheduler
    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    scaler = torch.cuda.amp.GradScaler()

    best_auc = 0
    early_stop_counter = 0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, scheduler, scaler, device)
        val_loss, val_labels, val_outputs = validate(model, val_loader, criterion, device)

        val_auc = roc_auc_score(val_labels, val_outputs)
        print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val AUC: {val_auc:.4f}")

        # Early Stopping
        if val_auc > best_auc:
            best_auc = val_auc
            torch.save(model.state_dict(), "best_model.pth")
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print("Early stopping triggered.")
            break

In [ ]:
def predict(csv_file, img_dir, model, img_size, batch_size, device):
    # Test dönüşümleri
    test_transform = A.Compose([
        A.Resize(img_size, img_size),
        ToTensorV2()
    ])

    # Dataset ve DataLoader oluşturma
    test_dataset = AdvancedImageDataset(csv_file, img_dir, transform=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Modeli yükle ve değerlendirme moduna al
    model.load_state_dict(torch.load("best_model.pth"))
    model.eval()
    predictions = []

    with torch.no_grad():
        for images, _ in tqdm(test_loader):
            # Görselleri uygun veri türüne dönüştür
            images = images.to(device, dtype=torch.float32)
            
            # Model tahminleri
            outputs = model(images)
            predictions.extend(torch.sigmoid(outputs).cpu().numpy())

    return predictions

## 10. Running the Code
> Use this section to train the model and make predictions.

In [ ]:
# Step 1: Load and map training labels
# Map the "editada" label to 0 (fake) and "real" to 1 (real)
labels = pd.read_csv("/kaggle/input/cidaut-ai-fake-scene-classification-2024/train.csv")
labels['label'] = labels['label'].map({"editada": 0, "real": 1})  # Binary classification labels

# Step 2: Define image directory and device
# Define the directory containing training images and check for GPU availability
img_dir = "/kaggle/input/cidaut-ai-fake-scene-classification-2024/Train"
model = AdvancedModel(num_classes=1)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Step 3: Train the model
# Train the advanced model with defined parameters
train_model(
    labels, 
    img_dir, 
    model, 
    img_size=224, 
    batch_size=32, 
    lr=1e-4, 
    num_epochs=20, 
    device=device, 
    patience=3
)

# Step 4: Load test labels
# Read the test file and prepare it for predictions
test_labels = pd.read_csv("/kaggle/input/cidaut-ai-fake-scene-classification-2024/sample_submission.csv")
test_img_dir = "/kaggle/input/cidaut-ai-fake-scene-classification-2024/Test"

# Step 5: Generate predictions
# Use the predict function to infer probabilities for test images
predictions = predict(
    test_labels, 
    test_img_dir, 
    model, 
    img_size=224, 
    batch_size=32, 
    device=device
)

# Step 6: Update test labels with predictions
# Add the predictions to the DataFrame and ensure the column is float type
test_labels['label'] = predictions
test_labels['label'] = test_labels['label'].astype(float)  # Ensure correct data type for submission

# Step 7: Save the submission file
# Save the test labels as submission.csv in the required format
test_labels.to_csv("submission.csv", index=False)

#GMM

In [ ]:
!pip install scikit-image==0.24.0


In [ ]:
!pip install nltk==3.2.4


In [ ]:
!pip install --upgrade autogluon


In [ ]:
import autogluon.multimodal as agmm
# from PIL import Image, ImageStat

In [ ]:
!pip install h2o


In [ ]:
import h2o
from h2o.automl import H2OAutoML

# Initialize H2O cluster
h2o.init()


In [ ]:
# Convert the NumPy arrays to a pandas DataFrame
X2_df = pd.DataFrame(X2)  # Assuming X2 is a NumPy array
# X2_df['label'] = y  # Add the target variable to the DataFrame

# Convert your Pandas DataFrame (X) and Series (y) into H2OFrames
X_h2o = h2o.H2OFrame(X2)
y_h2o = h2o.H2OFrame(pd.DataFrame(y))

# Add the target column ('label') to X
X_h2o['label'] = y_h2o

# Split the dataset into train and test sets
train, test = X_h2o.split_frame(ratios=[0.8], seed=1234)

# Specify the target column and features
target = 'label'
features = X_h2o.columns
features.remove(target)

# Train the AutoML model
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=features, y=target, training_frame=train)

# Get the leaderboard
lb = aml.leaderboard
print(lb)

# Make predictions on the test set
predictions = aml.predict(test)

# Optionally, evaluate the model
perf = aml.leader.model_performance(test)
print(perf)